In [586]:
import pandas as pd

In [664]:
votes = pd.read_csv("../sqlite/2020.06.16.csv")
votes["vote date"] = "06-16-2020"

In [665]:
votes.columns = votes.columns.str.lower()
votes.columns = votes.columns.str.replace("\n", " ")

In [666]:
votes.loc[79, "first name"] = "Thomas"
votes.loc[90, "first name"] = "Alexander"
votes.loc[18, "first name"] = "Judith"
votes.loc[38, "first name"] =  "Suzanne"
votes.loc[3, "first name"] =  "Stephan"
votes.loc[121, "first name"] = "Matthew"
votes.loc[157, "first name"] = "Julia"
votes.loc[247, "first name"] ="Philip"
votes.loc[103, "first name"] ="Nancy" 
votes.loc[196, "first name"] ="David" 

In [667]:
votes["middle name"] = votes["first name"].str.split(" ").apply(lambda x: " ".join(x[1:]) if len(x) > 1 else None)
votes["first name"] = votes["first name"].str.split(" ").apply(lambda x: x[0])

In [668]:
def remove_abv(name):
    if name:
        tokens = name.split(".")
        if len(tokens) > 1:
            return tokens[1]
        return tokens[0]
    
def preprocess_names(name_ser):
    name_ser = name_ser.str.lower()
    name_ser = name_ser.str.replace("\n", " ")
    name_ser = name_ser.str.replace("“", "\"")
    name_ser = name_ser.str.replace("”", "\"")
    name_ser = name_ser.str.split(",").apply(lambda x: x[0] if x else None)
    name_ser = name_ser.str.replace(r"\"(.*?)\"", "")
    name_ser = name_ser.str.replace("-", " ")
    name_ser = name_ser.apply(remove_abv)
    name_ser = name_ser.str.replace(r"\s+", " ")
    name_ser = name_ser.str.strip(" ")
    return name_ser

In [669]:
votes["first_name_clean"] = preprocess_names(votes["first name"])
votes["last_name_clean"] = preprocess_names(votes["last name"])
votes["middle_name_clean"] = preprocess_names(votes["middle name"])

In [670]:
votes.to_csv("../sqlite/2020.06.16_clean.csv", index=False)

In [671]:
years = pd.read_csv("../sqlite/years_served.csv")

In [672]:
years.columns = years.columns.str.lower()

In [673]:
years["first_name_clean"] = preprocess_names(years["first name"])
years["last_name_clean"] = preprocess_names(years["last name"])

In [674]:
# years["middle name"] = years["first name"].str.split(" ").apply(lambda x: " ".join(x[1:]) if len(x) > 1 else None)
years["first_name_clean"] = years["first_name_clean"].str.split(" ").apply(lambda x: x[0])

In [675]:
years.to_csv("../sqlite/years_served_clean.csv", index=False)

In [676]:
years

,precinct,first name,middle,last name,street num,street name,unit,total years served,seeking re-election 2021.04.06,first_name_clean,last_name_clean
0,1,Jennifer,G.,Ausrotas,65,RICHARDSON RD,NaN,1,True,jennifer,ausrotas
1,1,Heather,A.,Barr,180,SCHOOL ST,NaN,2,NaN,heather,barr
2,1,Daniel,Patrick,Barry,129,Goden St,NaN,6,NaN,daniel,barry
3,1,Jessica,Lee,Bennett,15,TROWBRIDGE ST,NaN,2,True,jessica,bennett
4,1,Carolyn,J.,Bishop,7,Orchard St,NaN,39,NaN,carolyn,bishop
...,...,...,...,...,...,...,...,...,...,...,...
282,At Large,Ellen,O'Brien,Cushman,38,Scott Rd,NaN,24,NaN,ellen,cushman
283,At Large,Adam,NaN,Dash,12,Goden St,NaN,12,NaN,adam,dash
284,At Large,Stephen,M.,Fiore,34,VAN NESS RD,NaN,0,NaN,stephen,fiore
285,At Large,Andrea,NaN,Prestwich,86,ALEXANDER AVE,NaN,4,NaN,andrea,prestwich


In [677]:
years[years["total years served"] > 0].shape[0] - votes.shape[0]

-5

In [678]:
merged = years[["first_name_clean", "last_name_clean", "precinct", "total years served"]].merge(votes[["first_name_clean", "last_name_clean", "pct"]], on=["first_name_clean", "last_name_clean"], how="left")

In [679]:
votes[votes["last_name_clean"].str.contains("goodman")]

,pct,first name,last name,attendance,votes available legend,remote town meeting vote,article 1: reports,consent agenda: articles 2-6,article 7: a) appropriation of capital expeditures,article 7: b) appropriation of capital expenditures,...,aricle 9: fy21 budget appropriation main motion,aricle 10: fy21 community preservation committee budget and projects,motion to terminate debate: discussion on article 11,article 11: community preservation committee amend 2018 appropriation for belmont housing,article 12: home rule petition for special act of the massachusetts legislature - remote participation of town meeting,vote date,middle name,first_name_clean,last_name_clean,middle_name_clean
130,3,Ariane,Goodman-\nBelkadi,Here,NaN,Yes / In Favor,Yes / In Favor,Yes / In Favor,Abstain,Abstain,...,Yes / In Favor,Yes / In Favor,Yes / In Favor,Yes / In Favor,Yes / In Favor,06-16-2020,None,ariane,goodman belkadi,None


In [680]:
votes.iloc[130].last_name_clean

'goodman belkadi'

In [681]:
len(merged[(merged["pct"].isnull()) & (merged["total years served"] > 0) ])

19

In [682]:
merged[(merged["pct"].isnull()) & (merged["total years served"] > 0) ]

,first_name_clean,last_name_clean,precinct,total years served,pct
8,peter,dizikes,1,5,NaN
14,jessica,hausman,1,2,NaN
18,gretchen,mcclain,1,8,NaN
26,jennifer,thomas,1,2,NaN
31,john,weis,1,11,NaN
37,patricia,brusch,2,33,NaN
57,john,murphy,2,1,NaN
87,ralph,jones,3,21,NaN
122,philip,lawrence,4,13,NaN
126,rose,o'neil,4,6,NaN
